In [8]:
import pandas as pd
import numpy as np
columns = [
    "Order ID",
    "Product",
    "Quantity Ordered",
    "Price Each",
    "Order Date",
    "Street",
    "City",
    "State Zip"
]

df = pd.read_csv("sales_data.csv",names=columns,header=0)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Street,City,State Zip
0,1001.0,USB-C Cable,2.0,11.95,01/01/23 10:00,917 1st St,Dallas,TX 75001
1,1002.0,Macbook Pro Laptop,1.0,1700.00,01/02/23 12:00,682 Chestnut St,Boston,MA 02215
2,1003.0,USB-C Cable,1.0,NaN,01/05/23 11:30,123 Main St,Austin,TX 73301
3,1004.0,AAA Batteries (4-pack),3.0,2.99,01/10/23 09:15,456 Elm St,San Francisco,CA 94016
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1005.0,AAA Batteries (4-pack),2.0,2.99,2023-02-01 14:00,789 Maple St,Los Angeles,CA 90001
6,1006.0,ThinkPad Laptop,1.0,999.99,02/05/23 16:20,321 Oak St,New York,NY 10001
7,1007.0,USB-C Cable,1.0,11.95,02/10/23 08:45,654 Pine St,Atlanta,GA 30301
8,1008.0,ThinkPad Laptop,1.0,NaN,02/15/23 13:00,987 Cedar St,Portland,OR 97035
9,1009.0,Lightning Cable,1.0,14.95,03/01/23 10:00,111 Birch St,Seattle,WA 98101


In [9]:
# 2. Remove duplicate header row
df = df[df["Order ID"] != "Order ID"]
df.reset_index(drop=True, inplace=True)
df.head()


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Street,City,State Zip
0,1001.0,USB-C Cable,2.0,11.95,01/01/23 10:00,917 1st St,Dallas,TX 75001
1,1002.0,Macbook Pro Laptop,1.0,1700.00,01/02/23 12:00,682 Chestnut St,Boston,MA 02215
2,1003.0,USB-C Cable,1.0,NaN,01/05/23 11:30,123 Main St,Austin,TX 73301
3,1004.0,AAA Batteries (4-pack),3.0,2.99,01/10/23 09:15,456 Elm St,San Francisco,CA 94016
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 3. Drop invalid rows
df = df.dropna(how="all")
df.shape

(11, 8)

In [11]:
# 4. Convert numeric columns
df["Order ID"] = pd.to_numeric(df["Order ID"])
df["Quantity Ordered"] = pd.to_numeric(df["Quantity Ordered"])
df["Price Each"] = pd.to_numeric(df["Price Each"])

df.dtypes

Order ID            float64
Product              object
Quantity Ordered    float64
Price Each          float64
Order Date           object
Street               object
City                 object
State Zip            object
dtype: object

In [12]:
#5.Fill Missing Price Values
price_mean = df.groupby("Product")["Price Each"].mean()

df["Price Each"] = df["Price Each"].fillna(
    df["Product"].map(price_mean)
)
df["Price Each"]

0       11.95
1     1700.00
2       11.95
3        2.99
5        2.99
6      999.99
7       11.95
8      999.99
9       14.95
10      14.95
11       2.99
Name: Price Each, dtype: float64

In [13]:
#6. Convert Order Date
df["Order Date"] = pd.to_datetime(df["Order Date"])
df[["Order Date"]].head()

C:\Users\sneha\AppData\Local\Temp\ipykernel_17924\2319454978.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Order Date"] = pd.to_datetime(df["Order Date"])


,Order Date
0,2023-01-01 10:00:00
1,2023-01-02 12:00:00
2,2023-01-05 11:30:00
3,2023-01-10 09:15:00
5,2023-02-01 14:00:00


In [14]:
#7. Extract Month
df["Month"] = df["Order Date"].dt.to_period("M")
df[["Order Date", "Month"]].head()

,Order Date,Month
0,2023-01-01 10:00:00,2023-01
1,2023-01-02 12:00:00,2023-01
2,2023-01-05 11:30:00,2023-01
3,2023-01-10 09:15:00,2023-01
5,2023-02-01 14:00:00,2023-02


In [15]:
#8. Calculate Total Sales
df["Total Sales"] = df["Quantity Ordered"] * df["Price Each"]
df[["Product", "Total Sales"]].head()

,Product,Total Sales
0,USB-C Cable,23.90
1,Macbook Pro Laptop,1700.00
2,USB-C Cable,11.95
3,AAA Batteries (4-pack),8.97
5,AAA Batteries (4-pack),5.98


In [20]:
# 9. Monthly Revenue
df["Month"] = df["Order Date"].dt.to_period("M")
monthly_revenue = df.groupby("Month")["Total Sales"].sum()
monthly_revenue


Month
2023-01    1744.82
2023-02    2017.91
2023-03      62.79
Freq: M, Name: Total Sales, dtype: float64

In [ ]:
# 10. Best Selling Product per Month
best_selling = (
    df.groupby(["Month", "Product"])["Quantity Ordered"].sum()
    .reset_index()
    .sort_values(["Month", "Quantity Ordered"], ascending=[True, False])
    .groupby("Month")
    .first()["Product"]
)

best_selling

Month
2023-01    AAA Batteries (4-pack)
2023-02    AAA Batteries (4-pack)
2023-03           Lightning Cable
Freq: M, Name: Product, dtype: object

In [18]:
df.to_csv("cleaned_sales.csv", index=False)